In [1]:
# junk for jupyter notebook
%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
#standard imports (I just copy and paste this into most notebooks)
import numpy as np
import matplotlib.pyplot as plt
import gmao_tools as gt
import matplotlib
#font = {'family' : 'normal',
#        'size'   : 18}

#matplotlib.rc('font', **font)

import simple_obsplot as sop
import cartopy.crs as ccrs

import ioda
import ioda_access as ia

In [47]:
config_dict = {
    'column_bins_min': gt.bins(1000,100,-100)[:-1]*100,
    'column_bins_max': gt.bins(1000,100,-100)[1:]*100,
    'column_variable': 'pres',
    'row_major_variable': 'kx'  ,
    'row_minor_variable': None,
    'row_major_bin': 'unique',
    'row_minor_bin': 'unique',
    'row_stat': ['count','mean','std','cpen'],
    'stat_var': 'omf'
}

In [48]:
from pprint import pprint

pprint(config_dict)

{'column_bins_max': array([95000., 85000., 75000., 65000., 55000., 45000., 35000., 25000.,
       15000.]),
 'column_bins_min': array([105000.,  95000.,  85000.,  75000.,  65000.,  55000.,  45000.,
        35000.,  25000.]),
 'column_variable': 'pres',
 'row_major_bin': 'unique',
 'row_major_variable': 'kx',
 'row_minor_bin': 'unique',
 'row_minor_variable': None,
 'row_stat': ['count', 'mean', 'std', 'cpen'],
 'stat_var': 'omf'}


In [49]:
#fn = '/discover/nobackup/drholdaw/JediEwok/R2D2DataStore/Local/geos/diag/a002/PT6H/2020-12-14/geos.a002.diag.PT6H.aircraft.2020-12-14T21:00:00Z.PT6H.nc4'
fn = '/discover/nobackup/projects/gmao/obsdev/wrmccart/jcsda/pyioda_test/geos.a005.diag.PT6H.aircraft.2020-12-15T03:00:00Z.PT6H.nc4'

acft_file = ia.obs(fn)
acft_file.set_obs_var('air_temperature')

In [103]:

def summary_table(data,config,in_msk=None):
    import pandas as pd
    
    if (in_msk is None):
        data.use_mask(False)
    else:
        data.set_mask(in_msk)
        data.use_mask(True)
    
    if (config['row_major_bin'] == 'unique'):
        row_major = np.unique(data.v( config['row_major_variable']))
    else:
        row_major = config['row_major_bin']
        
    df = pd.DataFrame()
    col_order = []
    
    init_loop = True
    
    for crow_major in row_major:
        major_msk = '({} == {})'.format(config['row_major_variable'], crow_major)
        doSub = config['row_minor_variable'] is not None
        # insert logic for subtypes here
        
        if doSub:
            #set mask here
            print('asdf')
        else:
            msk = major_msk
                        
        for stat in config['row_stat']:
            cdict = {config['row_major_variable']: crow_major ,
                     'stat': stat}
            if (init_loop): col_order.append(config['row_major_variable'])
            
            if config['row_minor_variable'] is not None: 
                cdict[ config['row_minor_variable'] ] = None
                if (init_loop): col_order.append( config['row_minor_variable'] )
                #update for minor var
                
            if (init_loop): col_order.append('stat')

            for cmin, cmax in zip(config['column_bins_min'],config['column_bins_max']):
                clabel = '{} - {}'.format(cmin,cmax)
                if (init_loop): col_order.append( clabel )
                col_msk = '({} <= {}) & ({} > {})'.format(config['column_variable'], cmin, config['column_variable'], cmax)
                total_msk = ''
                if (in_msk is not None):
                    total_msk = in_msk + ' & ' 
                total_msk = total_msk + major_msk + ' & ' + col_msk
                
#                print(total_msk)
                data.set_mask(total_msk)
                data.use_mask(True)
                
                cdict[clabel] = data.stat(stat, config['stat_var'])
                
            init_loop = False
            df = df.append(cdict, ignore_index = True)

                
#             cdf = pd.DataFrame(data=cdict)
            
#             if df is None:
#                 cdf = df
#             else:
#                 df = df.append(cdf)
    df = df[col_order]
    return(df)

In [110]:
base_msk = '(kx > 0)'
acft_file.set_mask(base_msk)
acft_file.use_mask(True)
df = summary_table(acft_file,config_dict,in_msk='(kx > 0)')

/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [111]:

# df.set_index(['kx','stat'])
df

,kx,stat,105000.0 - 95000.0,95000.0 - 85000.0,85000.0 - 75000.0,75000.0 - 65000.0,65000.0 - 55000.0,55000.0 - 45000.0,45000.0 - 35000.0,35000.0 - 25000.0,25000.0 - 15000.0
0,130.0,count,0.000000,8.000000,3.000000e+00,0.000000,3.000000,2.000000e+00,0.000000,110.000000,852.000000
1,130.0,mean,NaN,2.504377,1.609945e+00,NaN,0.461517,4.613747e+00,NaN,0.661126,1.288851
2,130.0,std,NaN,2.283104,2.713833e+00,NaN,0.737567,8.501663e-01,NaN,1.141934,1.524469
3,130.0,cpen,NaN,1.390070,1.001841e-73,NaN,1.182818,2.214560e-73,NaN,0.797844,0.961063
4,131.0,count,2179.000000,2070.000000,1.259000e+03,1159.000000,1109.000000,1.164000e+03,1152.000000,1228.000000,2655.000000
5,131.0,mean,0.576793,0.480104,2.590390e-01,0.147484,-0.049024,2.393591e-01,0.167452,0.248745,0.665192
6,131.0,std,1.975551,1.340060,1.138364e+00,0.959449,0.921984,9.535537e-01,0.894894,0.989918,1.248101
7,131.0,cpen,1.574483,1.404735,1.628386e+00,1.042528,1.056931,9.802456e-01,0.616556,0.503789,0.382295
8,133.0,count,2769.000000,5378.000000,3.478000e+03,4428.000000,3409.000000,3.050000e+03,2690.000000,4441.000000,12584.000000
9,133.0,mean,0.979752,0.576133,1.261115e-01,0.300830,0.143551,2.073111e-01,0.319707,0.416570,0.352668


In [112]:

df = df.set_index(['kx','stat'])


In [113]:
df

105000.0 - 95000.0  95000.0 - 85000.0  85000.0 - 75000.0  \
kx    stat                                                              
130.0 count            0.000000           8.000000       3.000000e+00   
      mean                  NaN           2.504377       1.609945e+00   
      std                   NaN           2.283104       2.713833e+00   
      cpen                  NaN           1.390070       1.001841e-73   
131.0 count         2179.000000        2070.000000       1.259000e+03   
      mean             0.576793           0.480104       2.590390e-01   
      std              1.975551           1.340060       1.138364e+00   
      cpen             1.574483           1.404735       1.628386e+00   
133.0 count         2769.000000        5378.000000       3.478000e+03   
      mean             0.979752           0.576133       1.261115e-01   
      std              2.867401           1.235630       1.147172e+00   
      cpen             1.195279           0.960651       1.332013e+00   
134.0 count          114.000000         216.000000       1.950000e+02   
      mean            -8.820165         -10.738280      -1.134883e+01   
      std             24.787380          25.954453       2.459447e+01   
      cpen             5.257751          14.439287       8.274938e+00   

             75000.0 - 65000.0  65000.0 - 55000.0  55000.0 - 45000.0  \
kx    stat                                                             
130.0 count           0.000000           3.000000       2.000000e+00   
      mean                 NaN           0.461517       4.613747e+00   
      std                  NaN           0.737567       8.501663e-01   
      cpen                 NaN           1.182818       2.214560e-73   
131.0 count        1159.000000        1109.000000       1.164000e+03   
      mean            0.147484          -0.049024       2.393591e-01   
      std             0.959449           0.921984       9.535537e-01   
      cpen            1.042528           1.056931       9.802456e-01   
133.0 count        4428.000000        3409.000000       3.050000e+03   
      mean            0.300830           0.143551       2.073111e-01   
      std             0.979640           0.843615       8.505864e-01   
      cpen            1.067721           0.807376       8.229063e-01   
134.0 count         221.000000         186.000000       1.820000e+02   
      mean           -8.364541          -8.450866      -8.305642e+00   
      std            21.146641          19.945539       2.171884e+01   
      cpen            5.213441           4.982817       5.816248e-01   

             45000.0 - 35000.0  35000.0 - 25000.0  25000.0 - 15000.0  
kx    stat                                                            
130.0 count           0.000000         110.000000         852.000000  
      mean                 NaN           0.661126           1.288851  
      std                  NaN           1.141934           1.524469  
      cpen                 NaN           0.797844           0.961063  
131.0 count        1152.000000        1228.000000        2655.000000  
      mean            0.167452           0.248745           0.665192  
      std             0.894894           0.989918           1.248101  
      cpen            0.616556           0.503789           0.382295  
133.0 count        2690.000000        4441.000000       12584.000000  
      mean            0.319707           0.416570           0.352668  
      std             0.817145           0.928447           1.316112  
      cpen            0.778805           0.335373           0.205353  
134.0 count         183.000000          71.000000          94.000000  
      mean           -6.183342          -5.271718          -2.657345  
      std            22.571074          24.842972          35.257492  
      cpen            0.694802           0.432175           0.103869

In [133]:
import ncdiag as ncd

t_file = ncd.obs('../x43_spire.diag_conv_t_ges.20200201_00z.nc4')

config_dict = {
    'column_bins_min': gt.bins(1000,100,-100)[:-1],
    'column_bins_max': gt.bins(1000,100,-100)[1:],
    'column_variable': 'pres',
    'row_major_variable': 'kx'  ,
    'row_minor_variable': None,
    'row_major_bin': 'unique',
    'row_minor_bin': 'unique',
    'row_stat': ['count','mean','std','cpen'],
    'stat_var': 'omf'
}

In [134]:

df = summary_table(t_file,config_dict)#,in_msk='(kx > 0)')
df

/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:1169: RuntimeWarning: overflow encountered in true_divide
  result = self.f(da, db, *args, **kwargs)
/discover/nobackup/wrmccart/python/ncdiag_functions.py:276: RuntimeWarning: divide by zero encountered in true_divide
  s = s + (comg / csigo)**2
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:5215: RuntimeWarning: Mean of empty slice.
  result = super(MaskedArray, self).mean(axis=axis,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:5293: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = super(MaskedArray, self).var(axis=axis, dtype=dtype, out=out,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:185: RuntimeWarning: invalid value encountered in

,kx,stat,1050.0 - 950.0,950.0 - 850.0,850.0 - 750.0,750.0 - 650.0,650.0 - 550.0,550.0 - 450.0,450.0 - 350.0,350.0 - 250.0,250.0 - 150.0
0,120.0,count,1373,1752,1960,1957,1432,2175,2104,2190,3542
1,120.0,mean,0.272336,0.257551,0.13049,0.123534,0.148367,0.00843884,-0.0725339,-0.27109,-0.158844
2,120.0,std,3.55797,1.91843,1.50807,1.23669,1.12816,0.956954,0.923355,1.09066,1.40345
3,120.0,cpen,--,2.48531,--,2.22013,2.95805,2.54913,2.33414,--,0.918128
4,126.0,count,344,888,541,35,0,0,0,0,0
5,126.0,mean,3.23343,2.84079,0.311151,-6.98875,NaN,NaN,NaN,NaN,NaN
6,126.0,std,5.79609,6.35122,2.21637,12.9067,--,--,--,--,--
7,126.0,cpen,4.40506e-07,4.84091e-07,5.00919e-08,2.15431e-06,NaN,NaN,NaN,NaN,NaN
8,130.0,count,6,42,49,29,37,19,46,382,1154
9,130.0,mean,6.61191,-0.113675,-0.379318,0.391327,-0.149656,2.91023,1.76326,0.210684,0.182797


In [135]:

df = df.set_index(['kx','stat'])
df

1050.0 - 950.0 950.0 - 850.0 850.0 - 750.0 750.0 - 650.0  \
kx    stat                                                             
120.0 count           1373          1752          1960          1957   
      mean        0.272336      0.257551       0.13049      0.123534   
      std          3.55797       1.91843       1.50807       1.23669   
      cpen              --       2.48531            --       2.22013   
126.0 count            344           888           541            35   
      mean         3.23343       2.84079      0.311151      -6.98875   
      std          5.79609       6.35122       2.21637       12.9067   
      cpen     4.40506e-07   4.84091e-07   5.00919e-08   2.15431e-06   
130.0 count              6            42            49            29   
      mean         6.61191     -0.113675     -0.379318      0.391327   
      std          5.16853       2.55018       3.01725        3.8168   
      cpen         0.01899     0.0150041     0.0310848     0.0508701   
131.0 count           3826          4780          2992          3014   
      mean        0.289396      0.836748      0.580974      0.440566   
      std           1.5516       1.53273       1.27731       1.16263   
      cpen        0.300901      0.386322      0.438463      0.385582   
132.0 count              9            10            13             0   
      mean        0.709504      0.326254       1.08324           NaN   
      std         0.386267      0.531238      0.761777            --   
      cpen       0.0554529      0.373951       2.00355           NaN   
133.0 count          12729         23253         16548         18237   
      mean        0.474264      0.158114      0.358353     0.0414225   
      std          2.25244       1.12223       1.02782      0.871964   
      cpen              --      0.586981      0.927276      0.673999   
134.0 count           2191          2677          1179          1337   
      mean        -3.82254      -3.43964      -3.15189      -4.03131   
      std           19.966       18.8607       19.2407       21.3827   
      cpen              --            --            --            --   
135.0 count            262           478           292           285   
      mean          1.4115       0.53228      0.296499       0.45166   
      std          1.69533      0.665316       0.89623      0.680529   
      cpen          0.1753     0.0239095   8.91157e-09    6.6713e-09   
180.0 count           7312             0             0             0   
      mean        0.591126           NaN           NaN           NaN   
      std          1.91239            --            --            --   
      cpen        0.248885           NaN           NaN           NaN   
181.0 count          17717          2970           723           182   
      mean          0.5445     0.0407343     -0.292642     -0.473716   
      std          2.76765       3.87898        3.6113       3.48148   
      cpen     7.95654e-08   1.50484e-07   1.31274e-07   1.23454e-07   
182.0 count              4             0             0             0   
      mean          1.1241           NaN           NaN           NaN   
      std        0.0694902            --            --            --   
      cpen        0.432828           NaN           NaN           NaN   
183.0 count           3003           529            42             0   
      mean        0.614124       0.73411        3.9503           NaN   
      std          2.43789       3.23844       3.08896            --   
      cpen     6.32059e-08   1.10266e-07    2.5147e-07           NaN   
187.0 count          38411          5194          1579           217   
      mean        0.539569      0.439066       1.23462       2.13864   
      std          2.16826       3.16871        3.7773       4.38963   
      cpen              --   1.02337e-07   1.57926e-07   2.38431e-07   
199.0 count             30             0             0             0   
      mean         -7.7419           NaN           N

In [136]:
gt.summary_table(t_file,config_dict)#,

/discover/nobackup/wrmccart/python/ncdiag_functions.py:276: RuntimeWarning: divide by zero encountered in true_divide
  s = s + (comg / csigo)**2
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:5215: RuntimeWarning: Mean of empty slice.
  result = super(MaskedArray, self).mean(axis=axis,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:5293: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = super(MaskedArray, self).var(axis=axis, dtype=dtype, out=out,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:206: RuntimeWarning: invalid value encountered in true

,kx,stat,1050.0 - 950.0,950.0 - 850.0,850.0 - 750.0,750.0 - 650.0,650.0 - 550.0,550.0 - 450.0,450.0 - 350.0,350.0 - 250.0,250.0 - 150.0
0,120.0,count,1373,1752,1960,1957,1432,2175,2104,2190,3542
1,120.0,mean,0.272336,0.257551,0.13049,0.123534,0.148367,0.00843884,-0.0725339,-0.27109,-0.158844
2,120.0,std,3.55797,1.91843,1.50807,1.23669,1.12816,0.956954,0.923355,1.09066,1.40345
3,120.0,cpen,--,2.48531,--,2.22013,2.95805,2.54913,2.33414,--,0.918128
4,126.0,count,344,888,541,35,0,0,0,0,0
5,126.0,mean,3.23343,2.84079,0.311151,-6.98875,NaN,NaN,NaN,NaN,NaN
6,126.0,std,5.79609,6.35122,2.21637,12.9067,--,--,--,--,--
7,126.0,cpen,4.40506e-07,4.84091e-07,5.00919e-08,2.15431e-06,NaN,NaN,NaN,NaN,NaN
8,130.0,count,6,42,49,29,37,19,46,382,1154
9,130.0,mean,6.61191,-0.113675,-0.379318,0.391327,-0.149656,2.91023,1.76326,0.210684,0.182797


In [168]:
#fn = '/discover/nobackup/drholdaw/JediEwok/R2D2DataStore/Local/geos/diag/a002/PT6H/2020-12-14/geos.a002.diag.PT6H.aircraft.2020-12-14T21:00:00Z.PT6H.nc4'
fn = '/discover/nobackup/projects/gmao/obsdev/wrmccart/jcsda/pyioda_test/geos.a005.diag.PT6H.aircraft.2020-12-15T03:00:00Z.PT6H.nc4'

acft_file = ia.obs(fn)
acft_file.set_obs_var('air_temperature')

config_dict = gt.yaml_to_dict('summary_table.example.yaml')

In [169]:
pprint(config_dict)

{'column_bins_max': [95000.0,
                     85000.0,
                     75000.0,
                     65000.0,
                     55000.0,
                     45000.0,
                     35000.0,
                     25000.0,
                     15000.0],
 'column_bins_min': [105000.0,
                     95000.0,
                     85000.0,
                     75000.0,
                     65000.0,
                     55000.0,
                     45000.0,
                     35000.0,
                     25000.0],
 'column_variable': 'pres',
 'row_major_bin': 'unique',
 'row_major_variable': 'kx',
 'row_minor_bin': 'unique',
 'row_minor_variable': None,
 'row_stat': ['count', 'mean', 'std', 'cpen'],
 'stat_var': 'omf'}


In [172]:

df = gt.summary_table(acft_file,config_dict,in_msk='(kx > 0)')#,verbose=True)

/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/fromnumeric.py:3334: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:161: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:216: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = _var(a, axis=axis, dtype=dtype, out=out, ddof=ddof,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:209: RuntimeWarning: invalid value encountered in true_divide
  ret = ret.dtype.type(ret / rcount)


In [174]:
df.fillna(0)

,kx,stat,105000.0 - 95000.0,95000.0 - 85000.0,85000.0 - 75000.0,75000.0 - 65000.0,65000.0 - 55000.0,55000.0 - 45000.0,45000.0 - 35000.0,35000.0 - 25000.0,25000.0 - 15000.0
0,130.0,count,0.000000,8.000000,3.000000e+00,0.000000,3.000000,2.000000e+00,0.000000,110.000000,852.000000
1,130.0,mean,0.000000,2.504377,1.609945e+00,0.000000,0.461517,4.613747e+00,0.000000,0.661126,1.288851
2,130.0,std,0.000000,2.283104,2.713833e+00,0.000000,0.737567,8.501663e-01,0.000000,1.141934,1.524469
3,130.0,cpen,0.000000,1.390070,1.001841e-73,0.000000,1.182818,2.214560e-73,0.000000,0.797844,0.961063
4,131.0,count,2179.000000,2070.000000,1.259000e+03,1159.000000,1109.000000,1.164000e+03,1152.000000,1228.000000,2655.000000
5,131.0,mean,0.576793,0.480104,2.590390e-01,0.147484,-0.049024,2.393591e-01,0.167452,0.248745,0.665192
6,131.0,std,1.975551,1.340060,1.138364e+00,0.959449,0.921984,9.535537e-01,0.894894,0.989918,1.248101
7,131.0,cpen,1.574483,1.404735,1.628386e+00,1.042528,1.056931,9.802456e-01,0.616556,0.503789,0.382295
8,133.0,count,2769.000000,5378.000000,3.478000e+03,4428.000000,3409.000000,3.050000e+03,2690.000000,4441.000000,12584.000000
9,133.0,mean,0.979752,0.576133,1.261115e-01,0.300830,0.143551,2.073111e-01,0.319707,0.416570,0.352668


In [202]:
import ncdiag as ncd

t_file = ncd.obs('../x43_spire.diag_conv_uv_ges.20200201_00z.nc4')

config_dict = {
    'column_bins_min': gt.bins(1000,100,-100)[:-1],
    'column_bins_max': gt.bins(1000,100,-100)[1:],
    'column_variable': 'pres',
    'row_major_variable': 'kx'  ,
    'row_minor_variable': None,
    'row_major_bin': 'unique',
    'row_minor_bin': 'unique',
    'row_stat': ['count','mean','std','cpen'],
    'stat_var': 'u_omf'
}


df = gt.summary_table(t_file,config_dict)#,in_msk='(kx > 0)')
df

/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:1169: RuntimeWarning: overflow encountered in true_divide
  result = self.f(da, db, *args, **kwargs)
/discover/nobackup/wrmccart/python/ncdiag_functions.py:276: RuntimeWarning: divide by zero encountered in true_divide
  s = s + (comg / csigo)**2
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:5215: RuntimeWarning: Mean of empty slice.
  result = super(MaskedArray, self).mean(axis=axis,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:5293: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = super(MaskedArray, self).var(axis=axis, dtype=dtype, out=out,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:185: RuntimeWarning: invalid value encountered in

,kx,stat,1050.0 - 950.0,950.0 - 850.0,850.0 - 750.0,750.0 - 650.0,650.0 - 550.0,550.0 - 450.0,450.0 - 350.0,350.0 - 250.0,250.0 - 150.0
0,220.0,count,1463,2196,2261,2006,1717,2118,2127,2469,3807
1,220.0,mean,-0.209496,0.0844679,0.198658,0.0463727,0.345577,0.331213,0.110375,0.295391,0.388068
2,220.0,std,2.54484,2.44772,2.41617,2.69724,2.60225,2.63385,2.90381,3.12952,3.43357
3,220.0,cpen,--,--,--,--,1.6166,--,--,--,--
4,221.0,count,108,160,116,54,32,29,14,19,18
...,...,...,...,...,...,...,...,...,...,...,...
111,287.0,cpen,2.72246e-08,4.76165e-08,6.7603e-08,1.09436e-07,3.04191e-08,NaN,NaN,NaN,NaN
112,290.0,count,17539,0,0,0,0,0,0,0,0
113,290.0,mean,-0.0573369,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
114,290.0,std,1.57008,--,--,--,--,--,--,--,--


In [201]:
config_dict = {
    'column_bins_min': gt.bins(1000,100,-100)[:-1],
    'column_bins_max': gt.bins(1000,100,-100)[1:],
    'column_variable': 'pres',
    'row_major_variable': 'kx'  ,
    'row_minor_variable': 'subtype',
    'row_major_bin': 'unique',
    'row_minor_bin': 'unique',
    'row_stat': ['count','mean','std','cpen'],
    'stat_var': 'u_omf'
}


df = gt.summary_table(t_file,config_dict)#,in_msk='(kx > 0)')
df

/discover/nobackup/wrmccart/python/ncdiag_functions.py:276: RuntimeWarning: divide by zero encountered in true_divide
  s = s + (comg / csigo)**2
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:5215: RuntimeWarning: Mean of empty slice.
  result = super(MaskedArray, self).mean(axis=axis,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:153: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/ma/core.py:5293: RuntimeWarning: Degrees of freedom <= 0 for slice
  ret = super(MaskedArray, self).var(axis=axis, dtype=dtype, out=out,
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:185: RuntimeWarning: invalid value encountered in true_divide
  arrmean = um.true_divide(
/home/wrmccart/.conda/envs/new4/lib/python3.8/site-packages/numpy/core/_methods.py:206: RuntimeWarning: invalid value encountered in true

,kx,subtype,stat,1050.0 - 950.0,950.0 - 850.0,850.0 - 750.0,750.0 - 650.0,650.0 - 550.0,550.0 - 450.0,450.0 - 350.0,350.0 - 250.0,250.0 - 150.0
0,220.0,0.0,count,1463,2196,2261,2006,1717,2118,2127,2469,3807
1,220.0,0.0,mean,-0.209496,0.0844679,0.198658,0.0463727,0.345577,0.331213,0.110375,0.295391,0.388068
2,220.0,0.0,std,2.54484,2.44772,2.41617,2.69724,2.60225,2.63385,2.90381,3.12952,3.43357
3,220.0,0.0,cpen,--,--,--,--,1.6166,--,--,--,--
4,221.0,0.0,count,108,160,116,54,32,29,14,19,18
...,...,...,...,...,...,...,...,...,...,...,...,...
191,290.0,3.0,cpen,--,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
192,290.0,4.0,count,8510,0,0,0,0,0,0,0,0
193,290.0,4.0,mean,-0.0681917,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
194,290.0,4.0,std,1.53435,--,--,--,--,--,--,--,--


In [183]:
t_file.set_mask('(kx > 0)')
print(np.unique(t_file.v('subtype')))

[0 1]
